In [ ]:
import pandas as pd
from datetime import datetime
dat = pd.read_csv('~/phone_data.csv')
dat['datetime'] = [datetime.strptime(d,'%d/%m/%y %H:%M') for d in dat['date']]
dat['date'] = [datetime.strptime(d,'%d/%m/%y %H:%M').date() for d in dat['date']]
#dat.set_index('index', inplace=True)
dat.drop(columns='index', inplace=True)
dat.head()


In [9]:
import numpy as np
np.random.random((2,3)).reshape(3,2)

array([[ 0.94599384,  0.01209504],
       [ 0.73431335,  0.46891071],
       [ 0.45516887,  0.71722762]])

In [ ]:
import pandas as pd
from datetime import datetime
dat = pd.read_csv('~/phone_data.csv')
dat['datetime'] = [ datetime.strptime(d, '%d/%m/%y %H:%M') for d in dat['date']]
dat['date']  = [ datetime.strptime(d, '%d/%m/%y %H:%M').date() for d in dat['date']]
dat.drop(columns = 'index', inplace=True)

In [ ]:
c = dat.groupby(['date','month']).agg({'duration':'sum'})
c.drop(index = 'date')

In [ ]:
c.sum(axis = 1)


In [ ]:
dat[dat.date == datetime.date(2014, 10, 15)]

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline


a = dat.pivot_table(index = 'month', columns='network', fill_value= 'duration', aggfunc = 'sum')



In [ ]:
### time series chart (key is convert index to datetime)
a = dat.groupby(['month','network'])['duration'].sum().unstack()

import matplotlib.pyplot as plt
a.index = pd.to_datetime(a.index)
a.plot(figsize=[10,5])
plt.show()

# histgram
import numpy as np
dat['duration'].quantile(np.linspace(0,1,11))

%matplotlib inline
pd.pivot_table(dat, values='duration', index = 'item',columns='network',aggfunc='mean').plot(kind ='bar')

### t-test two sample and one sample
dat.groupby('item')['duration'].mean()

from scipy import stats
a = dat[dat['item'] =='call']['duration']
b = dat[dat['item'] =='data']['duration']
stats.ttest_ind(a,b)
stats.ttest_1samp(a,1000)

##### data aggregation
# How many rows the dataset
len(dat)
830
# What was the longest phone call / data entry?
dat.groupby('item')['duration'].max()
# How many seconds of phone calls are recorded in total?

dat[dat['item'] =='call'].groupby('month')['duration'].sum()

# How many entries are there for each month?
dat['month'].value_counts().sort_index()

# Number of non-null unique network entries
dat['network'].dropna().nunique()
"9"
# Get the first entry for each month
dat.sort_values(by = ['month','date']).groupby('month').first()

# Get the sum of the durations per month
dat.groupby('month')['duration'].sum()
# Get the number of dates / entries in each month
dat.groupby('month')['date'].nunique()
# What is the sum of durations, for calls only, to each network
dat[dat['item'] == 'call'].groupby('network')['duration'].sum()
# How many calls, sms, and data entries are in each month?
dat.groupby(['month','item'])['date'].count()

# How many calls, texts, and data are sent per month, split by network_type?
dat.groupby(['month','network','item'])['date'].count()
# produces Pandas Series
#data.groupby('month')['duration'].sum() 
# Produces Pandas DataFrame
#data.groupby('month')[['duration']].sum()

#data.groupby('month', as_index=False).agg({"duration": "sum"})
# Group the data frame by month and item and extract a number of stats from each group
agg = {'duration':{'duration_min':'min','duration_max':'max'},
       'network': 'count',
        'network_type':'count'}
test = dat.groupby(['month','item']).agg(agg)

In [ ]:
import numpy as np
df = dat[['duration','item','network','network_type']]

Y=df['item']
X=df.drop(columns='item')
X = pd.get_dummies(X)

from sklearn.preprocessing import StandardScaler   
#StandardScaler (scale feature) - 
# vs Normalizer (scale each sample)  - 
# vs Binalizer (conver continuous data to binary based on a threhold)

#from sklearn.preprocessing import Binarizer
#binarizer = Binarizer(threshold = 0.7)
#binarizer.fit_transform(np.array([0,1,2,3,4])[:,np.newaxis])

#from sklearn.preprocessing import Normalizer
#scalor = Normalizer()
#scalor.fit_transform(np.array([0,1,2,3,4])[:,np.newaxis])

#scalor = StandardScaler().fit(X['duration'][:,np.newaxis])
#X_scaled = scalor.transform(X['duration'][:,np.newaxis])
#X_scaled2 =np.hstack((X_scaled, X.drop(columns ='duration').values))

scalor = StandardScaler().fit(X)
X_scaled2 = scalor.transform(X)
X_scaled2

#from sklearn.model_selection import train_test_split
#X_train, X_test, Y_train, Y_test = train_test_split(X_scaled2,Y, test_size = 0.2, random_state = 0)

from sklearn.model_selection import StratifiedKFold
eval_size = 0.1
skf = StratifiedKFold(n_splits=10)
skf.get_n_splits(X_scaled2,Y)

for train_index, test_index in skf.split(X_scaled2,Y):
    X_train, X_test = X_scaled2[train_index], X_scaled2[test_index]
    Y_train, Y_test = Y.values[train_index], Y.values[test_index]


In [ ]:
from sklearn.svm import SVC
model = SVC(kernel = 'linear')

In [ ]:
from sklearn.preprocessing import LabelEncoder
enc = LabelEncoder()
Ytrain = enc.fit_transform(Y_train)
Ytest = enc.fit_transform(Y_test)

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score

model = GaussianNB()
model.fit(X_train, Y_train)
Y_pred = model.predict(X_test)

#from sklearn.metrics import confusion_matrix
#display(confusion_matrix(Y_test, Y_pred, labels= np.unique(Y_train)))
# much better confusion matrix
print(pd.crosstab(Y_test, Y_pred, rownames= ['True'], colnames= ['Predicted']).apply(lambda r: 100.0 * r/r.sum()))
print(accuracy_score(Y_test, Y_pred))
print(classification_report(Y_test, Y_pred))

from sklearn.linear_model improt LinearRegression
model = LinearRegression(fit_intercept= True)
#from sklearn.neighbors import KNeighborsClassifier
# model = KNeighborsClassifier(n_neighbors = 5)

#from sklearn.ensemble import RandomForestClassifier
#model = RandomForestClassifier(n_estimators = 100, n_jobs=1,min_samples_leaf = 5)
# Supported criteria are “gini” for the Gini impurity and “entropy” for the information gain

#from sklearn.svm import SVC
#model = SVC(kernel = 'linear')

from sklearn.model_selection import cross_val_score
print(cross_val_score(model, X_train, Y_train, cv=4))

In [12]:
0.5**0

1.0

In [11]:
def factorial(m):
    f = 1
    for i in range(m):
        f = f* (i+1)
    return f
factorial(0)

1

In [ ]:
from sklearn.model_selection import cross_val_score
print(cross_val_score(model, X_train, Y_train, cv=4))


In [ ]:
import matplotlib.pyplot as plt
plt.style.use('classic')
x = np.linspace(0, 10, 1000)
fig, ax = plt.subplots()
ax.plot(x, np.sin(x), '-b', label='Sine')
ax.plot(x, np.cos(x), '--r', label='Cosine')
ax.axis('equal')
leg = ax.legend();
ax.legend(loc='upper left', frameon=False)
fig.show()

import pandas as pd
cities = pd.read_csv('data/california_cities.csv')

# Extract the data we're interested in
lat, lon = cities['latd'], cities['longd']
population, area = cities['population_total'], cities['area_total_km2']

# Scatter the points, using size and color but no label
plt.scatter(lon, lat, label=None,
            c=np.log10(population), cmap='viridis',
            s=area, linewidth=0, alpha=0.5)
plt.axis(aspect='equal')
plt.xlabel('longitude')
plt.ylabel('latitude')
plt.colorbar(label='log$_{10}$(population)')
plt.clim(3, 7)

# Here we create a legend:
# we'll plot empty lists with the desired size and label
for area in [100, 300, 500]:
    plt.scatter([], [], c='k', alpha=0.3, s=area,
                label=str(area) + ' km$^2$')
plt.legend(scatterpoints=1, frameon=False, labelspacing=1, title='City Area')

plt.title('California Cities: Area and Population');

In [ ]:
fig, ax = plt.subplots(2, 3, sharex='col', sharey='row')

# axes are in a two-dimensional array, indexed by [row, col]
for i in range(2):
    for j in range(3):
        ax[i, j].text(0.5, 0.5, str((i, j)),
                      fontsize=18, ha='center')
fig

In [ ]:
>>> df = pd.DataFrame(np.random.randn(10,3),
...                   columns=['Col1', 'Col2', 'Col3'])
>>> df['X'] = pd.Series(['A', 'A', 'A', 'A', 'A',
...                      'B', 'B', 'B', 'B', 'B'])
>>> df['Y'] = pd.Series(['A', 'B', 'A', 'B', 'A',
...                      'B', 'A', 'B', 'A', 'B'])
>>> boxplot = df.boxplot(column=['Col1', 'Col2'], by=['X', 'Y'])

In [ ]:
import pyodbc
import pandas as pd

conn = pyodbc.connect(r'Driver={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=C:\Users\Ron\Desktop\testdb.accdb;')

SQL_Query = pd.read_sql_query(
'''select
product_name,
product_price_per_unit,
units_ordered,
((units_ordered) * (product_price_per_unit)) AS revenue
from tracking_sales''', conn)

df = pd.DataFrame(SQL_Query, columns=['product_name','product_price_per_unit','units_ordered','revenue'])
print (df)

In [ ]:
pandas.DataFrame.from_records(
        run_query(ctx, qry=qry, output=True),
        columns=['non_pc_id', 'unique_cnt', 'country'])

In [ ]:
  cnx = snowflake.connector.connect(
           user=uid,
           password=pwd,
           account=account,
         )
    cursor = ctx.cursor()
    print(qry)
    cursor.execute(qry)
    outputs = cursor.fetchall()

In [ ]:
import numpy as np
np.random.randint(0,10,(3,3))